In [ ]:
import os
import json

import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

from scipy.interpolate import splev,splrep,interp1d

from sunkit_dem import Model
from sunkit_dem.models import single_gaussian

from sunkit_dem.util import quantity_1d_to_sequence

%matplotlib inline

In [ ]:
import ndcube
print(ndcube.__version__)

In [ ]:
temperature_bin_edges = 10.**np.arange(5.5,7.5,0.1) * u.K

log_temperature_bin_centers = (np.log10(temperature_bin_edges.value)[1:] + np.log10(temperature_bin_edges.value)[:-1])/2

temperature_bin_centers = (
    10**log_temperature_bin_centers
    * temperature_bin_edges.unit
)

In [ ]:
EM0_1 = 5e22 / (u.cm**5 * u.K)
logT0_1 = 6.6
delta_logT_1 = 0.2

In [ ]:
from astropy.modeling.models import Gaussian1D

g = Gaussian1D(amplitude=EM0_1, mean=logT0_1, stddev=delta_logT_1)

dem_true = g(log_temperature_bin_centers)

In [ ]:
fig = plt.figure(); ax = fig.gca()
ax.plot(temperature_bin_centers,dem_true, '--')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e20,1e23)
ax.set_xlim(10**(5.2),10**(7.5))
ax.set_xlabel(r'$T$ [K]');
ax.set_ylabel(r'$\mathrm{DEM}$ [cm$^{-5}$ K$^{-1}$]');

In [ ]:
channels = [94, 131,  171, 193, 211, 335]

In [ ]:
with open('/Users/pwright/Documents/personal/sunkit-dem/sdo_aia.json') as json_file:
    response_data = json.load(json_file)
    
response = {}
for c in channels:
    nots = splrep(response_data[f'{c}']['temperature_response_x'],
                  response_data[f'{c}']['temperature_response_y'])
    response[c] = u.Quantity(splev(temperature_bin_centers.value, nots),
                             response_data[f'{c}']['temperature_response_y_units'],)

In [ ]:
intensity = u.Quantity([
    (response[c] * dem_true  * np.diff(temperature_bin_edges)).sum()
    for c in channels
]).decompose()

In [ ]:
uncertainty = intensity * 0.2

In [ ]:
seq = quantity_1d_to_sequence(intensity,
                              channels*u.angstrom,
                              uncertainty=uncertainty)

In [ ]:
# Fit model

In [ ]:
sg = Model(seq, response, temperature_bin_edges, model='single_gaussian')

In [ ]:
dem = sg.fit()

In [ ]:
fig = plt.figure(); ax = fig.gca()
ax.plot(temperature_bin_centers,dem_true, '--', label='dem_original')
ax.plot(temperature_bin_centers, dem['dem'].data, '.', color='red', label='fitted')
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(1e20,1e23)
ax.set_xlim(10**(5.2),10**(7.5))
ax.set_xlabel(r'$T$ [K]')
ax.set_ylabel(r'$\mathrm{DEM}$ [cm$^{-5}$ K$^{-1}$]')
ax.legend()